    작성일 : 20/03/08
    작성자 : 김은주
    
    작성내용 : 무신사 상품 크롤링_최종프로젝트 모델링 반영
               -> 상품 랭킹에 따른 상품정보 크롤링
               -> 특정 카테고리 전체상품에 따른 상품정보 크롤링으로 DB모델링 변경
    
    수정내용 
        20/03/11
               코드 수정
               -> 크롤링 페이지 변동으로 인한 코드 수정
                  (카테고리별 랭킹 상품 -> 카테고리별 전체 상품)
               -> 오류 확인 코드 추가
               -> 기존에 담긴 상품 크롤링 예외처리 추가
    
    테이블생성 : 
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_TOP(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_OUTER(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_BOTTOM(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_SNEAKER(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    

In [1]:
import requests, pymysql, math, re, time
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
# ## DB 연결 데이터 
# config = {"host":"192.168.0.12", "user":"musinsa", "passwd":"1111", "db":"PR_TEST"}
# conn = pymysql.connect(**config)

In [8]:
## DB 연결 데이터 
config = {"host":"127.0.0.1", "user":"root", "passwd":"1111", "db":"musinsa_test"}
conn = pymysql.connect(**config)

categorys = {'상의':'001', '아우터':"002", '바지':'003', '가방':"004", '신발':'005', '시계':'006',
            '모자':'007', '레그웨어/속옷':'008', '안경':'009', '액세서리':'011', '디지털/테크':'012',
            '생활/취미/예술':'013', '책/음악/티켓':'014', '코스메틱':'015', '스포츠/용품':'017',
            '스니커':'018', '원피스':'020', '반려동물':'021', '스커트':'022'}
tables = {'001':'FINAL_TOP', '002':'FINAL_OUTER', '003':'FINAL_BOTTOM', '004':'FINAL_BAG',
          '005':'FINAL_SHOE', '006':'FINAL_WATCH', '007':'FINAL_HAT', '008':'FINAL_UNDER',
          '009':'FINAL_GLASS', '011':'FINAL_ACC', '012':'FINAL_DIGITAL', '013':'FINAL_ART',
          '014':'FINAL_BOOK', '015':'FINAL_COSMETIC', '017':'FINAL_SPORT', '018':'FINAL_SNEAKER',
          '020':'FINAL_ONEPIECE', '021':'FINAL_PET', '022':'FINAL_SKIRT'}

# html 파싱
def get_html(url):
    html = ""
    resp = requests.get(url)
    
    if resp.status_code == 200 :
        html = resp.text
        
    return html

# 기존 상품코드 중복제외 가져오기
def musinsa_pdrt_select(category):
    cursor = conn.cursor()
    
    sql = "SELECT DISTINCT prdt_code FROM "+tables[category]+";"
    cursor.execute(sql)
    codes = [] #상품코드 담기
    for i in cursor.fetchall() :
                codes.append(str(i[0]))
            
    return codes

### 전체 페이지 상품정보 크롤링_DB적재상품 중복제외 전

In [ ]:
# 카테고리별 전체상품 크롤링
def crawl_musinsa_total(url, category):
    startTime = time.time()
    
    brands = []       #브랜드 담기 위한 리스트
    products = []     #상품명, 링크, 코드 담기 위한 리스트
    
    html = get_html(url+"1") # 전체상품 리스트 중 1페이지 접근
    soup = BeautifulSoup(html)
    
    
    # 상품리스트 페이지 수 담기
    page_num = int(soup.find('span', class_="totalPagingNum").text.strip())

    
    # 상품리스트 페이지 반복문
    for page in range(1,page_num+1)[151:200]:
        html = get_html(url+str(page))
        soup = BeautifulSoup(html)
        
        # 페이지 내 상품별 반복문
        for li in soup('li', class_="li_box") :
            
            try:
                code = None
                
                #상품명, 상품링크, 상품코드 뽑기
                for product_p in li('p', class_="list_info") :
                    prdt_code = product_p.a['href'].replace("/app/product/detail/", "").replace("/0","")
                    code = prdt_code
                    products.append(["prdt_name", product_p.a.get_text().replace("...","").strip()])
                    products.append(["prdt_code", prdt_code])

                    ## PRDT_SEX 가져오기
                    html = get_html("https://store.musinsa.com/app/product/detail/"+prdt_code)
                    soup = BeautifulSoup(html)
                    for p in soup.find_all("p", class_="product_article_contents"):            
                        if p.find("span", class_="txt_gender") != None:
                            products.append(["prdt_sex", p.select("span>span")[0].text.strip()])
                dict_product = dict(products)
                
                #브랜드코드 뽑기
                for brand_p in li('p', class_="item_title") :
                    for brand_p2 in brand_p('a') :
                        brd_code = brand_p2.get('href').replace("/app/brand/goods_list/","").strip()
                        brands.append(["brd_code", brd_code])
                dict_brand = dict(brands)                      





                # DB 적재
                cursor = conn.cursor()
                sql = "INSERT INTO " + tables[category]
                data = [dict_product.get("prdt_code", "오류"),
                        dict_brand.get("brd_code", "오류"),
                        dict_product.get("prdt_name", "오류"),
                        dict_product.get("prdt_sex", '오류')]

                cursor.execute(sql+"(PRDT_CODE, BRD_CODE, PRDT_NAME, PRDT_SEX) \
                               VALUES(%s, %s, %s, %s)", data)

                conn.commit()
            
            
            except:
                print("{}페이지 {}상품에서 오류발생".format(page, dict_product.get("prdt_code", "오류")))

            
        print("{}페이지 완료 : {}".format(page, tables[category]))
    print("--- {} seconds ---".format(time.time() - startTime))

In [ ]:
c = categorys["상의"]

url = "https://store.musinsa.com/app/items/lists/{}/?category=&d_cat_cd={}&u_cat_cd=&brand=&sort=pop&sub_sort=&display_cnt=90&page="

# 함수 실행
crawl_musinsa_total(url.format(c, c), c)

# conn.close

### 전체 페이지 상품정보 크롤링_DB적재상품 중복제외 후

In [9]:
# 카테고리별 전체상품 크롤링
def crawl_musinsa_total(url, category, codes):
    startTime = time.time()
    
    brands = []       #브랜드 담기 위한 리스트
    products = []     #상품명, 링크, 코드 담기 위한 리스트
    
    html = get_html(url+"1") # 전체상품 리스트 중 1페이지 접근
    soup = BeautifulSoup(html)
    
    
    # 상품리스트 페이지 수 담기
    page_num = int(soup.find('span', class_="totalPagingNum").text.strip())

    
    # 상품리스트 페이지 반복문
    for page in range(1,page_num+1)[172:200]:
        html = get_html(url+str(page))
        soup = BeautifulSoup(html)
        
        
        # 페이지 내 상품별 반복문
        for li in soup('li', class_="li_box") :
            
            try:
                # 상품코드 담기
                for product_p in li('p', class_="list_info") :
                    prdt_code = product_p.a['href'].replace("/app/product/detail/", "").replace("/0","")
                
                # 기존 DB에 있는 상품 거르기
                if prdt_code not in codes:
                    
                    # 상품링크, 상품코드 뽑기
                    for product_p in li('p', class_="list_info") :
                        products.append(["prdt_name", product_p.a.get_text().replace("...","").strip()])
                        products.append(["prdt_code", prdt_code])

                        ## PRDT_SEX 가져오기
                        html = get_html("https://store.musinsa.com/app/product/detail/"+prdt_code)
                        soup = BeautifulSoup(html)
                        for p in soup.find_all("p", class_="product_article_contents"):            
                            if p.find("span", class_="txt_gender") != None:
                                products.append(["prdt_sex", p.select("span>span")[0].text.strip()])
                    dict_product = dict(products)

                    #브랜드코드 뽑기
                    for brand_p in li('p', class_="item_title") :
                        for brand_p2 in brand_p('a') :
                            brd_code = brand_p2.get('href').replace("/app/brand/goods_list/","").strip()
                            brands.append(["brd_code", brd_code])
                    dict_brand = dict(brands)
                    

                    # DB 적재
                    cursor = conn.cursor()
                    sql = "INSERT INTO " + tables[category]
                    data = [dict_product.get("prdt_code", "오류"),
                            dict_brand.get("brd_code", "오류"),
                            dict_product.get("prdt_name", "오류"),
                            dict_product.get("prdt_sex", '오류')]

                    cursor.execute(sql+"(PRDT_CODE, BRD_CODE, PRDT_NAME, PRDT_SEX) \
                                   VALUES(%s, %s, %s, %s)", data)

                    conn.commit()
            
            
            except:
                print("{}페이지 {}상품에서 오류발생".format(page, dict_product.get("prdt_code", "오류")))

            
        print("{}페이지 완료 : {}".format(page, tables[category]))
    print("--- {} seconds ---".format(time.time() - startTime))

In [10]:
c = categorys["상의"]

url = "https://store.musinsa.com/app/items/lists/{}/?category=&d_cat_cd={}&u_cat_cd=&brand=&sort=pop&sub_sort=&display_cnt=90&page="


# 함수 실행
crawl_musinsa_total(url.format(c, c), c, musinsa_pdrt_select(c))

# conn.close

173페이지 완료 : FINAL_TOP
174페이지 완료 : FINAL_TOP
175페이지 완료 : FINAL_TOP
176페이지 완료 : FINAL_TOP
177페이지 완료 : FINAL_TOP
178페이지 완료 : FINAL_TOP
179페이지 완료 : FINAL_TOP
180페이지 완료 : FINAL_TOP
181페이지 완료 : FINAL_TOP
182페이지 완료 : FINAL_TOP
183페이지 완료 : FINAL_TOP
184페이지 완료 : FINAL_TOP
185페이지 완료 : FINAL_TOP
186페이지 완료 : FINAL_TOP
187페이지 완료 : FINAL_TOP
188페이지 완료 : FINAL_TOP
189페이지 완료 : FINAL_TOP
190페이지 완료 : FINAL_TOP
191페이지 완료 : FINAL_TOP
192페이지 완료 : FINAL_TOP
193페이지 완료 : FINAL_TOP
194페이지 완료 : FINAL_TOP
195페이지 완료 : FINAL_TOP
196페이지 완료 : FINAL_TOP
197페이지 완료 : FINAL_TOP
198페이지 완료 : FINAL_TOP
199페이지 완료 : FINAL_TOP
200페이지 완료 : FINAL_TOP
--- 2208.8658034801483 seconds ---


### 랭킹 페이지 상품정보 크롤링

In [ ]:
# 카테고리 랭킹 크롤링
def crawl_musinsa_rank(url_front, url_mid, url_end, category):
    startTime = time.time()
    ranks = []        #랭크 담기 위한 리스트
    brands = []       #브랜드명 담기 위한 리스트
    products = []     #상품명, 링크, 코드 담기 위한 리스트
    
    html = get_html(url_front+category+url_mid+"1"+url_end)
    soup = BeautifulSoup(html)
    
    
    # 랭킹페이지 수 100개 제한
    # 카테고리 랭킹 내 상품 수 (총 상품 수 / 80 올림)
    goods_cnt = math.ceil(int(str.strip(soup.find('span', class_="counter box_num_goods").text.replace(',','').replace('상품 수 : ','').replace('개',''))) / 80)
    if goods_cnt > 100:
        rankPage = list(range(1, 101))
    else:
        rankPage = list(range(1, goods_cnt+1))
    
    # 랭크 페이지 반복문
    for page_range in rankPage[:1]:
        html = get_html(url_front+category+url_mid+str(page_range)+url_end)
        soup = BeautifulSoup(html)
        

        temp= [] #랭크 html소스 담기
        for li in soup('li', class_="li_box") :

            #랭크 뽑기
            for rank_p in li('p', class_="n-label") :
                temp = rank_p.get_text().replace("\t","").replace("\n","").strip().split("위")
                ranks.append(["prdt_rank", temp[0]])
            dict_rank = dict(ranks)
            
            
            #브랜드코드 뽑기
            for brand_p in li('p', class_="item_title") :
                for brand_p2 in brand_p('a') :
                    brd_code = brand_p2.get('href').replace("/app/brand/goods_list/","").strip()
                    brands.append(["brd_code", brd_code])
            dict_brand = dict(brands)                      
            
            
            #상품명, 상품링크, 상품코드 뽑기
            for product_p in li('p', class_="list_info") :
                prdt_code = product_p.a['href'].replace("/app/product/detail/", "").replace("/0","")
                products.append(["prdt_name", product_p.a.get_text().strip()])
                products.append(["prdt_code", prdt_code])
                
                ## PRDT_SEX 가져오기
                html = get_html("https://store.musinsa.com/app/product/detail/"+prdt_code)
                soup = BeautifulSoup(html)
                for p in soup.find_all("p", class_="product_article_contents"):            
                    if p.find("span", class_="txt_gender") != None:
                        products.append(["prdt_sex", p.select("span>span")[0].text.strip()])
            dict_product = dict(products)
            
            
            # DB 적재
            cursor = conn.cursor()
            sql = "INSERT INTO " + tables[category]
            data = [dict_product.get("prdt_code", "오류"),
                    dict_brand.get("brd_code", "오류"),
                    dict_product.get("prdt_name", "오류"),
                    dict_rank.get("prdt_rank", '오류'),
                    dict_product.get("prdt_sex", '오류')]

            cursor.execute(sql+"(PRDT_CODE, BRD_CODE, PRDT_NAME, PRDT_RANK, PRDT_SEX) \
                           VALUES(%s, %s, %s, %s, %s)", data)        
            
            conn.commit()     
        print("--- {} seconds ---".format(time.time() - startTime))
        print("{}페이지 완료 : {}".format(page_range, tables[category]))

In [ ]:
c = categorys["상의"]

url_front = "https://store.musinsa.com/app/contents/bestranking/?d_cat_cd=&u_cat_cd="
url_mid = "&range=1d&price=&ex_soldout=&sale_goods=&new_product_yn=&list_kind=&page="
url_end = "&display_cnt=80&sex=&popup=&sale_rate=&price1=&price2=&chk_new_product_yn=&chk_sale=&chk_soldout="

# 함수 실행
crawl_musinsa_rank(url_front, url_mid, url_end, c)

conn.close